In [5]:
import numpy as np
import open3d as o3d
import os
import copy

In [13]:
def load_landmarks(txt_path):
    """
    Load landmark coordinates exported from CloudCompare.
    Supports both CSV (x,y,z) and space-separated formats.
    Returns: (N,3) numpy array
    """
    pts = []
    with open(txt_path, "r") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue

            if "," in line:
                vals = line.split(",")
            else:
                vals = line.split()

            if len(vals) < 3:
                continue

            try:
                x = float(vals[0])
                y = float(vals[1])
                z = float(vals[2])
                pts.append([x, y, z])
            except ValueError:
                continue

    return np.asarray(pts, dtype=np.float64)

def compute_rigid_transform(P, Q):
    """
    Given 2 list of points:
      P: (N, 3) moving object points (mov_landmarks)
      Q: (N, 3) reference object points (ref_landmarks)
    Solve Rigid transformation T so that Q ≈ R @ P + t
    Return:
     4x4 transform matrix T
    """
    assert P.shape == Q.shape
    # Centroid
    centroid_P = P.mean(axis=0)
    centroid_Q = Q.mean(axis=0)

    # Decentroid
    P_centered = P - centroid_P
    Q_centered = Q - centroid_Q

    # Covariance
    H = P_centered.T @ Q_centered

    # SVD
    U, S, Vt = np.linalg.svd(H)
    R = Vt.T @ U.T

    # Reflection case
    if np.linalg.det(R) < 0:
        Vt[2, :] *= -1
        R = Vt.T @ U.T

    # translation
    t = centroid_Q - R @ centroid_P

    T = np.eye(4)
    T[:3, :3] = R
    T[:3, 3] = t
    return T


In [14]:
print(os.getcwd())

/Users/tuboshu/Documents/2025/M1/PRAT/Code


In [15]:
ref_pcd_path = "../Data/ICP_test/ref_Tete_D.ply"
mov_pcd_path = "../Data/ICP_test/move_Dragon_01_Transform.ply"
ref_lm_path  = "../Data/ICP_test/picking_list_Tete_D.txt"
mov_lm_path  = "../Data/ICP_test/picking_list_Dragon_01_Transform.txt"

# Read Point Cloud
ref_pcd = o3d.io.read_point_cloud(ref_pcd_path)
mov_pcd_raw = o3d.io.read_point_cloud(mov_pcd_path)

print(ref_pcd)
print(mov_pcd_raw)

# Read landmarks
Q = load_landmarks(ref_lm_path)  # reference
P = load_landmarks(mov_lm_path)  # moving

print("Landmark pairs:", P.shape)


PointCloud with 1349372 points.
PointCloud with 1711170 points.
Landmark pairs: (5, 3)


In [16]:
T_init = compute_rigid_transform(P, Q)
print("Initial transform (from landmarks):\n", T_init)
np.savetxt("T_landmark_4x4.txt", T_init)


# mov_pcd_landmark = mov_pcd.transform(T_init.copy())

# o3d.io.write_point_cloud("../Data/ICP_test/dragon_after_landmark.ply", mov_pcd_landmark)

mov_after_landmark = copy.deepcopy(mov_pcd_raw)
mov_after_landmark.transform(T_init)
o3d.io.write_point_cloud("../Data/ICP_test/dragon_after_landmark.ply", mov_after_landmark)



Initial transform (from landmarks):
 [[ 9.62467722e-01  2.70397248e-01 -2.32639644e-02 -3.62131506e+01]
 [-2.70754231e-01  9.50764689e-01 -1.50793408e-01 -8.51340723e+02]
 [-1.86555667e-02  1.51432605e-01  9.88291524e-01 -2.97968502e+01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


True

In [17]:
max_corr_dist = 10.0

result_icp = o3d.pipelines.registration.registration_icp(
    mov_pcd_raw,                   # source
    ref_pcd,                       # target
    max_corr_dist,
    T_init,                        # init_trans from Landmark coarse registration
    o3d.pipelines.registration.TransformationEstimationPointToPoint(),
    o3d.pipelines.registration.ICPConvergenceCriteria(
        max_iteration=100
    )
)

print("ICP result:")
print("  fitness:", result_icp.fitness)
print("  inlier_rmse:", result_icp.inlier_rmse)
print("  T_icp:\n", result_icp.transformation)


ICP result:
  fitness: 0.8167061133610337
  inlier_rmse: 1.7607851593499075
  T_icp:
 [[ 9.54561581e-01  2.96914653e-01 -2.55710273e-02 -4.80688633e+01]
 [-2.97020892e-01  9.40874457e-01 -1.62892129e-01 -9.31408116e+02]
 [-2.43059335e-02  1.63085697e-01  9.86312464e-01 -4.09150237e+01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


In [18]:
T_total = result_icp.transformation
np.savetxt("T_icp_total_4x4.txt", T_total)

mov_after_icp = copy.deepcopy(mov_pcd_raw)
mov_after_icp.transform(T_total)
o3d.io.write_point_cloud("../Data/ICP_test/dragon_after_icp_python.ply", mov_after_icp)


print("Saved aligned point cloud to dragon_after_icp_python.ply")
print("Saved ICP transform to T_icp_4x4.txt")


Saved aligned point cloud to dragon_after_icp_python.ply
Saved ICP transform to T_icp_4x4.txt


### Comparison
Apply icp to mov dragon

[11:24:13] [ComputeDistances] Mean distance = 11.4888 / std deviation = 26.862

Default software algo:

[11:58:09] [ComputeDistances] Mean distance = 11.5154 / std deviation = 26.9277

### FPFH + RANSAC

In [ ]:
def preprocess_point_cloud(pcd: o3d.geometry.PointCloud, voxel_size):
    """
    Downsample + normal estimation + FPFH feature extraction.

    Returns:
        pcd_down: downsampled point cloud with normals
        fpfh:     o3d.pipelines.registration.Feature (shape: 33 x N)
    """
    # Voxel downsample 
    pcd_down = pcd.voxel_down_sample(voxel_size)

    # Remove statistical outliers
    # Considered as outlier if avg dist among neighbors >= global avg dist + std_ratio * std
    pcd_down, _ = pcd_down.remove_statistical_outlier(nb_neighbors=20, std_ratio=2.0)

    # Estimate normals
    radius_normal = voxel_size * 2.0
    pcd_down.estimate_normals(
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normal, max_nn=30)
    )

    # Compute FPFH feature
    radius_feature = voxel_size * 5.0
    fpfh = o3d.pipelines.registration.compute_fpfh_feature(
        pcd_down,
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius_feature, max_nn=100)
    )

    return pcd_down, fpfh


def draw_registration_result(source, target, transformation):
    """Visualize alignment with two colors."""
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.paint_uniform_color([1.0, 0.0, 0.0])  # red
    target_temp.paint_uniform_color([0.0, 0.5, 1.0])  # blue
    source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target_temp])


def global_registration_ransac(src_down, tgt_down, src_fpfh, tgt_fpfh, voxel_size: float):
    """
    RANSAC-based global registration on FPFH feature matches.
    """
    distance_threshold = voxel_size * 1.5

    result = o3d.pipelines.registration.registration_ransac_based_on_feature_matching(
        src_down, tgt_down,
        src_fpfh, tgt_fpfh,
        mutual_filter=True, # ensure src/tar NN corespondance
        max_correspondence_distance=distance_threshold,
        estimation_method=o3d.pipelines.registration.TransformationEstimationPointToPoint(False), # rigid transform
        ransac_n=4, # sample point pairs to solve T 
        checkers=[ # pre filter
            # Enforce similar edge length ratios to reject degenerate matches
            o3d.pipelines.registration.CorrespondenceCheckerBasedOnEdgeLength(0.9),
            # Reject matches with too large geometric distance
            o3d.pipelines.registration.CorrespondenceCheckerBasedOnDistance(distance_threshold),
        ],
        criteria=o3d.pipelines.registration.RANSACConvergenceCriteria(
            max_iteration=100000,  
            confidence=0.999
        )
    )
    return result


def refine_registration_icp(src_full, tgt_full, init_T, voxel_size, use_point_to_plane=False):
    """
    ICP refinement (point-to-plane is usually better if normals are reliable).
    """
    # Set a tighter threshold than RANSAC
    max_corr_dist = voxel_size * 1.0

    if use_point_to_plane:
        # Normals are required for point-to-plane ICP (target normals are used)
        tgt_full.estimate_normals(
            o3d.geometry.KDTreeSearchParamHybrid(radius=voxel_size * 2.0, max_nn=30)
        )
        src_full.estimate_normals(
            o3d.geometry.KDTreeSearchParamHybrid(radius=voxel_size * 2.0, max_nn=30)
        )
        estimation = o3d.pipelines.registration.TransformationEstimationPointToPlane()
    else:
        estimation = o3d.pipelines.registration.TransformationEstimationPointToPoint()

    result_icp = o3d.pipelines.registration.registration_icp(
        src_full, tgt_full,
        max_corr_dist,
        init_T,
        estimation_method=estimation,
        criteria=o3d.pipelines.registration.ICPConvergenceCriteria(max_iteration=80)
    )
    return result_icp



In [3]:

ref_path = "../Data/ICP_test/ref_Tete_D.ply"
mov_path = "../Data/ICP_test/move_Dragon_01_Transform.ply"

ref_pcd = o3d.io.read_point_cloud(ref_path)
mov_pcd = o3d.io.read_point_cloud(mov_path)

print("Loaded:")
print("  ref:", ref_pcd)
print("  mov:", mov_pcd)


voxel_size = 5.0

# 1) Preprocess for FPFH + RANSAC on downsampled point clouds
ref_down, ref_fpfh = preprocess_point_cloud(ref_pcd, voxel_size)
mov_down, mov_fpfh = preprocess_point_cloud(mov_pcd, voxel_size)

print("Downsampled:")
print("  ref_down:", ref_down)
print("  mov_down:", mov_down)
print("FPFH dims:", ref_fpfh.data.shape, mov_fpfh.data.shape)  # (33, N)



Loaded:
  ref: PointCloud with 1349372 points.
  mov: PointCloud with 1711170 points.
Downsampled:
  ref_down: PointCloud with 57450 points.
  mov_down: PointCloud with 59566 points.
FPFH dims: (33, 57450) (33, 59566)


In [6]:
# 2) Global registration (RANSAC)
result_ransac = global_registration_ransac(mov_down, ref_down, mov_fpfh, ref_fpfh, voxel_size)
print("\nRANSAC result:")
print("  fitness:", result_ransac.fitness)
print("  inlier_rmse:", result_ransac.inlier_rmse)
print("  T_ransac:\n", result_ransac.transformation)

# Visualize coarse alignment
draw_registration_result(mov_down, ref_down, result_ransac.transformation)




RANSAC result:
  fitness: 0.7830305879192828
  inlier_rmse: 2.664491435886807
  T_ransac:
 [[ 9.55147585e-01  2.95106543e-01 -2.46011912e-02 -4.31887167e+01]
 [-2.95069755e-01  9.41413079e-01 -1.63325612e-01 -9.31767843e+02]
 [-2.50385735e-02  1.63259131e-01  9.86265444e-01 -4.12658185e+01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


In [7]:
# 3) ICP refinement on full-resolution clouds (or you can use downsampled first)
result_icp = refine_registration_icp(mov_pcd, ref_pcd, result_ransac.transformation, voxel_size, use_point_to_plane=True)
print("\nICP result:")
print("  fitness:", result_icp.fitness)
print("  inlier_rmse:", result_icp.inlier_rmse)
print("  T_icp:\n", result_icp.transformation)

# Visualize refined alignment
draw_registration_result(mov_pcd, ref_pcd, result_icp.transformation)




ICP result:
  fitness: 0.7907022680388273
  inlier_rmse: 1.2211446515516153
  T_icp:
 [[ 9.54329399e-01  2.97668491e-01 -2.54728865e-02 -4.74875924e+01]
 [-2.97744996e-01  9.40625150e-01 -1.63009950e-01 -9.32482530e+02]
 [-2.45624882e-02  1.63149612e-01  9.86295538e-01 -4.11525286e+01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


In [8]:
# 4) Save transformed moving point cloud (optional)
mov_aligned = copy.deepcopy(mov_pcd)
mov_aligned.transform(result_icp.transformation)
o3d.io.write_point_cloud("../Data/FPFH/mov_aligned_by_fpfh_ransac_icp_p2p.ply", mov_aligned)
print("\nSaved:", "mov_aligned_by_fpfh_ransac_icp_p2p.ply")


Saved: mov_aligned_by_fpfh_ransac_icp_p2p.ply


### Comparison
[16:12:34] [ComputeDistances] Mean distance = 11.5045 / std deviation = 26.9021

### Evaluation 

In [9]:

def nn_distances(source: o3d.geometry.PointCloud, target: o3d.geometry.PointCloud) -> np.ndarray:
    """
    Compute nearest neighbor distances from each point in 'source' to 'target'.
    Returns:
        dists: (N,) array of Euclidean distances.
    """
    target_kd = o3d.geometry.KDTreeFlann(target)
    src_pts = np.asarray(source.points)

    dists = np.empty(len(src_pts), dtype=np.float64)
    for i, p in enumerate(src_pts):
        # 1-NN search
        _, idx, dist2 = target_kd.search_knn_vector_3d(p, 1)
        dists[i] = np.sqrt(dist2[0])
    return dists


def registration_metrics(source_aligned: o3d.geometry.PointCloud,
                         target: o3d.geometry.PointCloud,
                         thresholds=(5.0, 10.0),
                         percentiles=(50, 90, 95)) -> dict:
    """
    Compute overlap-aware metrics for registration evaluation.

    Metrics:
      - median / P90 / P95 distances
      - coverage@tau: ratio of points with dist < tau
      - trimmed_rmse@tau: RMSE computed only on points with dist < tau
      - mean / std (for reference)
    """
    d = nn_distances(source_aligned, target)

    out = {
        "mean": float(d.mean()),
        "std": float(d.std()),
        "median": float(np.percentile(d, 50)),
    }
    for p in percentiles:
        out[f"p{p}"] = float(np.percentile(d, p))

    for tau in thresholds:
        inliers = d < tau
        coverage = float(inliers.mean())
        if inliers.any():
            trimmed_rmse = float(np.sqrt(np.mean(d[inliers] ** 2)))
            trimmed_mean = float(d[inliers].mean())
        else:
            trimmed_rmse = float("nan")
            trimmed_mean = float("nan")
        out[f"coverage@{tau}"] = coverage
        out[f"trimmed_mean@{tau}"] = trimmed_mean
        out[f"trimmed_rmse@{tau}"] = trimmed_rmse

    return out


def symmetric_chamfer(source_aligned: o3d.geometry.PointCloud,
                      target: o3d.geometry.PointCloud) -> float:
    """
    Symmetric Chamfer distance (mean NN distance both directions).
    """
    d_st = nn_distances(source_aligned, target).mean()
    d_ts = nn_distances(target, source_aligned).mean()
    return float(d_st + d_ts)

In [10]:
ref_path = "../Data/ICP_test/ref_Tete_D.ply"
mov_path = "../Data/ICP_test/move_Dragon_01_Transform.ply"
src = o3d.io.read_point_cloud(mov_path)
tgt = o3d.io.read_point_cloud(ref_path)

# src_aligned = copy.deepcopy(src)
# src_aligned.transform(T_total)

metrics = registration_metrics(src, tgt, thresholds=(5.0, 10.0))
print("Original not registered evaluation")
print(metrics)

print("Symmetric Chamfer:", symmetric_chamfer(src, tgt))

Original not registered evaluation
{'mean': 28.790589237437796, 'std': 21.611957785388448, 'median': 24.060411166186718, 'p50': 24.060411166186718, 'p90': 61.29547679266786, 'p95': 71.37153082536858, 'coverage@5.0': 0.1196292595124973, 'trimmed_mean@5.0': 2.4777668386423106, 'trimmed_rmse@5.0': 2.83796814333311, 'coverage@10.0': 0.22594189940216344, 'trimmed_mean@10.0': 4.844387508719292, 'trimmed_rmse@10.0': 5.637451842177725}
Symmetric Chamfer: 54.16179091590028


In [19]:
ref_path = "../Data/ICP_test/ref_Tete_D.ply"
mov_path = "../Data/ICP_test/dragon_after_icp_python.ply"
src = o3d.io.read_point_cloud(mov_path)
tgt = o3d.io.read_point_cloud(ref_path)

# src_aligned = copy.deepcopy(src)
# src_aligned.transform(T_total)

metrics = registration_metrics(src, tgt, thresholds=(5.0, 10.0))
print("Manual landmark + ICP evaluation")
print(metrics)

print("Symmetric Chamfer:", symmetric_chamfer(src, tgt))

Manual landmark + ICP evaluation
{'mean': 11.900873394379946, 'std': 26.68532721206653, 'median': 0.8916812282821849, 'p50': 0.8916812282821849, 'p90': 52.385716913856236, 'p95': 79.81466731250123, 'coverage@5.0': 0.7907560324222608, 'trimmed_mean@5.0': 0.9568073840270434, 'trimmed_rmse@5.0': 1.224656404608553, 'coverage@10.0': 0.8167061133610337, 'trimmed_mean@10.0': 1.1507352380483076, 'trimmed_rmse@10.0': 1.7607851593499064}
Symmetric Chamfer: 16.668003987122248


In [12]:
ref_path = "../Data/ICP_test/ref_Tete_D.ply"
mov_path = "../Data/FPFH/mov_aligned_by_fpfh_ransac_icp_p2p.ply"
src = o3d.io.read_point_cloud(mov_path)
tgt = o3d.io.read_point_cloud(ref_path)

# src_aligned = copy.deepcopy(src)
# src_aligned.transform(T_total)

metrics = registration_metrics(src, tgt, thresholds=(5.0, 10.0))
print("FPFH + RANSAC + ICP evaluation")
print(metrics)

print("Symmetric Chamfer:", symmetric_chamfer(src, tgt))

FPFH + RANSAC + ICP evaluation
{'mean': 11.919651183532961, 'std': 26.723596740054912, 'median': 0.8878449859671382, 'p50': 0.8878449859671382, 'p90': 52.503750839545056, 'p95': 79.93291604220614, 'coverage@5.0': 0.7907022680388273, 'trimmed_mean@5.0': 0.953115089069285, 'trimmed_rmse@5.0': 1.2211446515516156, 'coverage@10.0': 0.8165477421880936, 'trimmed_mean@10.0': 1.1466224471712876, 'trimmed_rmse@10.0': 1.7575508328904914}
Symmetric Chamfer: 16.678766929614426


### SpinNet

In [3]:
import numpy as np
import open3d as o3d
import torch
import torch.nn.functional as F
import sys, os
sys.path.insert(0, os.path.abspath("./SpinNet"))
from network.SpinNet import Descriptor_Net

def build_spinnet_model(ckpt_path: str,
                        des_r=0.30, rad_n=9, azi_n=80, ele_n=40,
                        voxel_r=0.04, voxel_sample=30,
                        dataset='3DMatch',
                        device='cuda'):
    model = Descriptor_Net(des_r, rad_n, azi_n, ele_n, voxel_r, voxel_sample, dataset)
    sd = torch.load(ckpt_path, map_location=device)
    model.load_state_dict(sd)
    model.eval()
    model.to(device)
    return model

def spinnet_features_for_pcd(pcd_down: o3d.geometry.PointCloud,
                             model: torch.nn.Module,
                             patch_radius: float,
                             N: int = 2048,
                             batch_size: int = 64,
                             device: str = 'cuda'):
    """
    For each point in pcd_down, build a local patch (N,3), run SpinNet, return Open3D Feature (32, num_pts).
    """
    pts = np.asarray(pcd_down.points).astype(np.float32)
    num_pts = pts.shape[0]

    # KDTree for neighborhood query
    kdtree = o3d.geometry.KDTreeFlann(pcd_down)

    desc_list = []
    # Build patches in chunks
    patches_buf = []
    idx_buf = []

    for i in range(num_pts):
        center = pts[i]

        # radius search
        _, idxs, _ = kdtree.search_radius_vector_3d(center, patch_radius)
        if len(idxs) < 5:
            # fallback: if neighborhood is too small, just repeat center
            patch = np.repeat(center[None, :], N, axis=0)
        else:
            neigh = pts[np.asarray(idxs, dtype=np.int64)]

            # sample to fixed N
            if neigh.shape[0] >= N:
                sel = np.random.choice(neigh.shape[0], N, replace=False)
                patch = neigh[sel]
            else:
                # pad by resampling with replacement
                sel = np.random.choice(neigh.shape[0], N, replace=True)
                patch = neigh[sel]

            # IMPORTANT: ensure the last point is the center (SpinNet uses input[:, -1, :] as center)
            patch[-1] = center

        patches_buf.append(patch)
        idx_buf.append(i)

        # run a batch
        if len(patches_buf) == batch_size or i == num_pts - 1:
            batch = torch.from_numpy(np.stack(patches_buf, axis=0)).to(device=device, dtype=torch.float32)  # (B,N,3)
            with torch.no_grad():
                out = model(batch)                   # (B,32,1,1)
                out = out.view(out.shape[0], -1)     # (B,32)
                out = F.normalize(out, p=2, dim=1)   # (B,32)
            desc_list.append(out.detach().cpu().numpy())
            patches_buf = []
            idx_buf = []

    desc = np.concatenate(desc_list, axis=0)  # (num_pts, 32)

    feat = o3d.pipelines.registration.Feature()
    feat.data = desc.T  # Open3D expects (dim, num_pts)
    return feat


ModuleNotFoundError: No module named 'pointnet2_ops'

In [ ]:
import open3d as o3d

ref_path = "../Data/ICP_test/ref_Tete_D.ply"
mov_path = "../Data/ICP_test/move_Dragon_01_Transform.ply"

ref_pcd = o3d.io.read_point_cloud(ref_path)
mov_pcd = o3d.io.read_point_cloud(mov_path)

bbox = ref_pcd.get_axis_aligned_bounding_box()
diag = np.linalg.norm(bbox.get_extent())
print("diag =", diag) # Verify the len unity

# 0) 统一单位：假设输入是 mm -> m
scale = 1e-3
ref_pcd.scale(scale, center=ref_pcd.get_center())
mov_pcd.scale(scale, center=mov_pcd.get_center())

# 1) downsample（单位：m）
voxel_size = 0.005  # 5mm = 0.005m
ref_down = ref_pcd.voxel_down_sample(voxel_size)
mov_down = mov_pcd.voxel_down_sample(voxel_size)

ref_down, _ = ref_down.remove_statistical_outlier(nb_neighbors=20, std_ratio=2.0)
mov_down, _ = mov_down.remove_statistical_outlier(nb_neighbors=20, std_ratio=2.0)

# 2) SpinNet model（你在 Mac 上就用 cpu）
ckpt_path = "pre-trained_models/3DMatch_best.pkl"
model = build_spinnet_model(
    ckpt_path,
    des_r=0.30, rad_n=9, azi_n=80, ele_n=40,
    voxel_r=0.04, voxel_sample=30,
    dataset="3DMatch",
    device="cpu"
)

# 3) SpinNet feature（patch_radius：m）
ref_feat = spinnet_features_for_pcd(ref_down, model, patch_radius=0.30, N=2048, batch_size=16, device="cpu")
mov_feat = spinnet_features_for_pcd(mov_down, model, patch_radius=0.30, N=2048, batch_size=16, device="cpu")

# 4) RANSAC + ICP（voxel_size：m）
result_ransac = global_registration_ransac(mov_down, ref_down, mov_feat, ref_feat, voxel_size)
result_icp = refine_registration_icp(mov_pcd, ref_pcd, result_ransac.transformation, voxel_size, use_point_to_plane=False)


In [ ]:
# 1) downsample 仍然保留（RANSAC/ICP 都需要）


ref_path = "../Data/ICP_test/ref_Tete_D.ply"
mov_path = "../Data/ICP_test/move_Dragon_01_Transform.ply"

ref_pcd = o3d.io.read_point_cloud(ref_path)
mov_pcd = o3d.io.read_point_cloud(mov_path)


voxel_size = 5.0
ref_down = ref_pcd.voxel_down_sample(voxel_size)
mov_down = mov_pcd.voxel_down_sample(voxel_size)

# 可选：去外点/估计法向（ICP point-to-plane 会用到）
ref_down, _ = ref_down.remove_statistical_outlier(nb_neighbors=20, std_ratio=2.0)
mov_down, _ = mov_down.remove_statistical_outlier(nb_neighbors=20, std_ratio=2.0)

# 2) SpinNet model
ckpt_path = "pre-trained_models/3DMatch_best.pkl"   # 你仓库里的路径
model = build_spinnet_model(ckpt_path,
                            des_r=0.30, rad_n=9, azi_n=80, ele_n=40,
                            voxel_r=0.04, voxel_sample=30,
                            dataset="3DMatch",
                            device="cuda")

# 3) 用 SpinNet 算 feature（返回 Open3D Feature）
# 注意：patch_radius 需要和你的点云单位一致。
# 如果你的数据是 mm，并且 voxel_size=5.0(mm)，那 des_r=0.30(m) 就不对：
#   - 要么把点云缩放到米
#   - 要么把 patch_radius 改成 300.0(mm)
ref_feat = spinnet_features_for_pcd(ref_down, model, patch_radius=300.0, N=2048, batch_size=64, device="cuda")
mov_feat = spinnet_features_for_pcd(mov_down, model, patch_radius=300.0, N=2048, batch_size=64, device="cuda")

# 4) RANSAC + ICP（你的函数可以不动，只是特征换了）
result_ransac = global_registration_ransac(mov_down, ref_down, mov_feat, ref_feat, voxel_size)
result_icp = refine_registration_icp(mov_pcd, ref_pcd, result_ransac.transformation, voxel_size, use_point_to_plane=False)
